### Import require library first

In [22]:
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import pandas as pd
from datetime import datetime
import os
import os.path
import sys
import pickle
import pprint
import time

import urllib.request, urllib.parse, urllib.error
from urllib import request as urlrequest

# proxies = {"http": "http://127.0.0.1:8080",
#            "https": "http://127.0.0.1:8080"}

max_try = 3

html_cache = '/Users/moming2k/project/TradingProjects/HKHorseDB/data/cache'
save_path = '/Users/chris.chan/project/TradingProjects/HKHorseDB/data/race_records'

# http://squidman.net/squidman/
    
#create the object, assign it to a variable
# proxy = urllib.request.ProxyHandler({'http': 'http://127.0.0.1:8080/','https': 'http://127.0.0.1:8080/'})
# # construct a new opener using your proxy settings
# opener = urllib.request.build_opener(proxy)
# # install the openen on the module-level
# urllib.request.install_opener(opener)

In [5]:
tmp_dict = {'HorseNo': None,
'HorseName': None,
'Age': None,
'JockeyName': None,
'ActualWt': None,
'Draw': None,
'Rate': None,
'Owner': None,
'DeclarHorseWt': None,
'WinOddBeforeNight': None,
'WinOddBeforeGate': None,             
'WinOdd': None,
'WinInTenThousand': None,
'PInTenThousand': None,
'P_Odd': None,
'PlaceInMiddle': None,
'Place': None,
'TimeInMiddle': None,
'FinishTime': None,
'LBW': None
}

In [6]:
def convert_str_to_int(int_str):
    try:
        return int(int_str)
    except Exception as err:
        return int_str

def convert_str_to_float(int_str):
    try:
        return float(int_str)
    except Exception as err:
        return int_str

In [7]:
def get_cache_path(url):
    url_path = urllib.parse.quote(url).replace('/', '_')
    file_path = "{}/{}".format(html_cache, url_path)
    return file_path

demo_url = get_cache_path("http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=05")
print(demo_url)

def is_cache_html(url):
    filepath = get_cache_path(url)
    if(os.path.isfile(filepath)):
        return True
    else:
        return False

def set_cache_html(url, html):
    filepath = get_cache_path(url)
    with open(filepath, 'w') as out:
        out.write(html)

def get_cache_html(url):
    filepath = get_cache_path(url)
    if(os.path.isfile(filepath)):
        print("url = {} exist in cache".format(url))
        with open(filepath, 'r') as io_file:
            html = io_file.read()        
        return html
    else:
        print("url = {} not exist in cache".format(url))
        return None
    

set_cache_html("http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=05", "abcdef")
get_cache_html("http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=05")

/Users/moming2k/project/TradingProjects/HKHorseDB/data/cache/http%3A__hk.racing.nextmedia.com_fullresult.php%3Fdate%3D20130206%26page%3D05
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=05 exist in cache


'abcdef'

In [10]:
def get_param_url(url, data_list=None):
    if data_list:
        url = "{}?{}".format(url, urllib.parse.urlencode(data_list))
    return url

def get(url, data_list=None, max_try=3):
    
    url = get_param_url(url, data_list)
        
    file_path = get_cache_path(url)
    
    query = urllib.request.Request(url)
    current_try = 0
    while current_try < max_try:
        try:
            if(current_try == 0):
                proxy = 'http://127.0.0.1:8080'
                os.environ['http_proxy'] = proxy 
                response = urllib.request.urlopen(query)
            else:
                os.environ['http_proxy'] = '' 
                response = urllib.request.urlopen(query)

            html = response.read()
            if(html != None):
#                 print('check html={}'.format(html))
                response.close()
                return html
            else:
                print('retry')
                current_try = current_try + 1
        except Exception as e:
            print('exception = {}'.format(e))
            current_try = current_try + 1
    raise Exception("Cannot open page {}".format(url))

In [47]:
# url = 'http://www.hkhorsedb.com/cseh/passodds.php'
# html = get(url)
# soup = BeautifulSoup(str(html, 'big5'), "html.parser")

In [12]:
def get_race_date_array():
    url = 'http://www.hkhorsedb.com/cseh/passodds.php'
    
    if is_cache_html(url):
        html = get_cache_html(url)
    else:
        html = get(url)
        html = str(html, 'big5')
        set_cache_html(url, html)
        
    soup = BeautifulSoup(html, "html.parser")
    tr_list = soup.findAll('table')[15].findAll('tr')[1:]
    
    race_date_array = []
    
    for i in range(0, len(tr_list)-1):
        tr = tr_list[i]
        tds = tr.findAll('td')
        
        for j in range(0, len(tds)):
            detail_date = tds[j].text
            
            if (detail_date != "-"):
                normal_data = ''.join(reversed(detail_date.split('-')))
                
                if(normal_data != ""):
                    race_date_array.append(normal_data)
    
    return race_date_array
    
race_date_array = get_race_date_array()
print(race_date_array[0:10])
print(race_date_array[-10:])

['20170625', '20170621', '20170618', '20170614', '20170611', '20170607', '20170604', '20170531', '20170528', '20170524']
['20051120', '20051116', '20051113', '20051109', '20051105', '20051101', '20051030', '20051026', '20051022', '20051019']


In [49]:
# url = 'http://www.hkhorsedb.com/cseh/passodds.php'
# html = get(url)
# soup = BeautifulSoup(str(html, 'big5'), "html.parser")

In [50]:
# tr_list = soup.findAll('table')[15].findAll('tr')[1:]

In [51]:
# print(tr_list[1])

In [52]:
# tr = tr_list[0]
# tds = tr.findAll('td')

In [53]:
# tds

In [54]:
# detail_date = tds[0].text
# detail_date

In [55]:
# date_info = ''.join(detail_date.split('-'))
# normal_data = ''.join(reversed(detail_date.split('-')))
# normal_data

In [56]:
# 'http://racing.hkjc.com/racing/Info/meeting/Results/english/Local/20130206/HV/5'

In [57]:
# tr_last = tr_list[-5]
# tds_last = tr_last.findAll('td')
# tds_last

In [29]:
def acquire_total_race(date_str):
    url = 'http://hk.racing.nextmedia.com/fullresult.php?date={}&page=01'.format(date_str)

    if is_cache_html(url):
        html = get_cache_html(url)
    else:
        html = get(url)
        html = str(html, 'utf-8')
        set_cache_html(url, html)
        
    soup = BeautifulSoup(html, "html.parser")
    try:
        return len(soup.findAll('table')[1].findAll('tr')[0].findAll('td')[0].findAll('a', href=True))
    except Exception as err:
        return -1

In [59]:
# urllib.parse.parse_qs(soup.findAll('table')[1].findAll('tr')[0].findAll('td')[0].findAll('a', href=True)[1]['href'])['page'][0]

In [14]:
def acquire_appledaily_race_record(date_str, race_id):
    url = 'http://hk.racing.nextmedia.com/fullresult.php?date={}&page={}'.format(date_str,race_id)
    # html = get(url)
    # soup = BeautifulSoup(str(html, 'utf-8'), "html.parser")
    
    if is_cache_html(url):
        html = get_cache_html(url)
    else:
        html = get(url)
        html = str(html, 'utf-8')
        set_cache_html(url, html)
    
    soup = BeautifulSoup(html, "html.parser")
        
    record_trs = soup.findAll('table')[2].findAll('tr')[2:-1]
    return record_trs

In [15]:
trs = acquire_appledaily_race_record("20051130","05")

url = http://hk.racing.nextmedia.com/fullresult.php?date=20051130&page=05 exist in cache


In [29]:
# next_result = 'http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=05'

In [63]:
# html_next = get(next_result)

In [64]:
# soup_next = BeautifulSoup(str(html_next, 'utf-8'), "html.parser")

In [65]:
# next_tr_list = soup_next.findAll('table')[2].findAll('tr')[2:-1]
# next_tr_list[0]

In [66]:
# next_tr_list[-1]

In [67]:
# next_td_list = next_tr_list[0].findAll('td')

In [68]:
# next_td_list

In [69]:
# '馬號	馬名	歲	騎師	負磅	檔	評分	廄	馬匹體重	賠率	獨贏票(萬)	位置票(萬)	位置賠率	走位	名次	分段時間	總時間	勝負距離'

In [70]:
# '7	超新鮮	4	薛寶力	123	1	67	姚	1121	4.8	3.7	4.5	455	346	1.5	2 2	1	12.75 21.61 23.01	(0.57.37)	頭馬'

In [16]:
appledaily_web_columns = ['Date','RaceNumber','HorseNo','HorseName','Age','JockeyName','ActualWt', 'Draw', 'Rate','Owner','DeclarHorseWt','WinOddBeforeNight','WinOddBeforeGate','WinOdd','WinInTenThousand','PInTenThousand','P_Odd','PlaceInMiddle','Place', 'TimeInMiddle','FinishTime','LBW']

In [17]:
tmp_result = tmp_dict.copy()

In [18]:
# setup the data array for later use


def process_get_race_date_id(date_str, race_id): #, driver, race_course='ST'):
    
    result_df = pd.DataFrame(columns=appledaily_web_columns)
    
    filename = '{}_{}.xlsx'.format(date_str, race_id)
    
    failed_info = ''
    
    next_tr_list = acquire_appledaily_race_record(date_str,race_id)
    try:
        for index in range(0, len(next_tr_list)-1):
            tmp_result = tmp_dict.copy()

#             print(index)

            next_td_list = next_tr_list[index].findAll('td')
            next_td_list

            try:
                tmp_result['Date'] = date_str
            except: 
                print('Can not find Date')
            
            try:
                tmp_result['RaceNumber'] = race_id
            except: 
                print('Can not find Race Number')
            
            
            try:
                tmp_result['HorseNo'] = convert_str_to_int(next_td_list[0].text)
            except IndexError:
                print('Can not find HorseNo')

            try:
                tmp_result['HorseName'] = next_td_list[1].text
            except IndexError:
                print('Can not find HorseName')
                continue

            try:
                tmp_result['Age'] = convert_str_to_int(next_td_list[2].text)
            except IndexError:
                print('Can not find Age')

            try:
                tmp_result['JockeyName'] = next_td_list[3].text
            except IndexError:
                print('Can not find JockeyName')

            try:
                tmp_result['ActualWt'] = convert_str_to_int(next_td_list[4].text)
            except IndexError:
                print('Can not find ActualWt')

            try:
                tmp_result['Draw'] = convert_str_to_int(next_td_list[5].text)
            except IndexError:
                print('Can not find Draw')

            try:
                tmp_result['Rate'] = convert_str_to_int(next_td_list[6].text)
            except IndexError:
                print('Can not find Rate')

            try:
                tmp_result['Owner'] = next_td_list[7].text
            except IndexError:
                print('Can not find Owner')

            try:
                tmp_result['DeclarHorseWt'] = convert_str_to_int(next_td_list[8].text)
            except IndexError:
                print('Can not find DeclarHorseWt')

            try:
                tmp_result['WinOddBeforeNight'] = convert_str_to_float(next_td_list[9].text)
            except IndexError:
                print('Can not find WinOddBeforeNight')

            try:
                tmp_result['WinOddBeforeGate'] = convert_str_to_float(next_td_list[10].text)
            except IndexError:
                print('Can not find WinOddBeforeGate')

            try:
                tmp_result['WinOdd'] = convert_str_to_float(next_td_list[11].text)
            except IndexError:
                print('Can not find WinOdd')

            try:
                tmp_result['WinInTenThousand'] = convert_str_to_float(next_td_list[12].text)
            except IndexError:
                print('Can not find WinInTenThousand')

            try:
                tmp_result['PInTenThousand'] = convert_str_to_float(next_td_list[13].text)
            except IndexError:
                print('Can not find PInTenThousand')

            try:
                tmp_result['P_Odd'] = convert_str_to_float(next_td_list[14].text)
            except IndexError:
                print('Can not find P_Odd')

            try:
                tmp_result['PlaceInMiddle'] = next_td_list[15].text
            except IndexError:
                print('Can not find PlaceInMiddle')

            try:
                tmp_result['Place'] = convert_str_to_int(next_td_list[16].text)
            except IndexError:
                print('Can not find Place')

            if(len(next_td_list) == 20):
                try:
                    tmp_result['TimeInMiddle'] = next_td_list[17].text
                except IndexError:
                    print('Can not find TimeInMiddle')

                try:
                    tmp_result['FinishTime'] = next_td_list[18].text
                except IndexError:
                    print('Can not find FinishTime')

                try:
                    tmp_result['LBW'] = next_td_list[19].text
                except IndexError:
                    print('Can not find LBW')
            else:
                try:
                    tmp_result['FinishTime'] = next_td_list[17].text
                except IndexError:
                    print('Can not find FinishTime')

                try:
                    tmp_result['LBW'] = next_td_list[18].text
                except IndexError:
                    print('Can not find LBW')

            result_df.loc[index] = tmp_result
            
        result_df.to_excel(os.path.join(save_path, '{}_{}.xlsx'.format(date_str, race_id)), index=False)
        result_df.to_csv(os.path.join(save_path, '{}_{}.cvs'.format(date_str, race_id)), index=False)

    except Exception as err:
        failed_info = '{} {}: test_failed as {}'.format(failed_info, datetime.today().strftime('%Y-%m-%d'), err)
        print(failed_info)

In [19]:
process_get_race_date_id("20051130","05")

url = http://hk.racing.nextmedia.com/fullresult.php?date=20051130&page=05 exist in cache


In [14]:
# result_df

NameError: name 'result_df' is not defined

In [76]:
# Add Leading Zero
# str(1).zfill(2)

In [77]:
# ~/Library/Logs/squid/squid-access.log

In [30]:
after = "20130324"
skip_now = True

for race_date in race_date_array:
    if(after == race_date):
        skip_now = False
    
    if(skip_now == True):
        print('skip for date = {}'.format(race_date))
    else:
        count = acquire_total_race(race_date)
        for index in range(1, count+1):
            try:
                race_index = str(index).zfill(2)
                url = 'http://hk.racing.nextmedia.com/fullresult.php?date={}&page={}'.format(race_date, race_index)
                print(url)

                is_cache = False
                if (is_cache_html(url)):
                    is_cache = True

                process_get_race_date_id(race_date, race_index)

                if (is_cache == True):
                    time.sleep(0.5)
                else:
                    time.sleep(3)

            except Exception as err:
                print('Error for date = {} and race = {}'.format(race_date,index))
                time.sleep(1)

skip for date = 20170625
skip for date = 20170621
skip for date = 20170618
skip for date = 20170614
skip for date = 20170611
skip for date = 20170607
skip for date = 20170604
skip for date = 20170531
skip for date = 20170528
skip for date = 20170524
skip for date = 20170521
skip for date = 20170517
skip for date = 20170513
skip for date = 20170510
skip for date = 20170507
skip for date = 20170503
skip for date = 20170430
skip for date = 20170426
skip for date = 20170423
skip for date = 20170420
skip for date = 20170417
skip for date = 20170412
skip for date = 20170409
skip for date = 20170405
skip for date = 20170402
skip for date = 20170329
skip for date = 20170326
skip for date = 20170322
skip for date = 20170319
skip for date = 20170315
skip for date = 20170312
skip for date = 20170308
skip for date = 20170305
skip for date = 20170301
skip for date = 20170226
skip for date = 20170222
skip for date = 20170219
skip for date = 20170215
skip for date = 20170211
skip for date = 20170208


http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=03
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=03 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=04
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=04 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=05
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=05 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=06
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=06 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=07
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=07 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=08
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130324&page=08 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=2013032

http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=03
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=03 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=04
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=04 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=05
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=05 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=06
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=06 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=07
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=07 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=08
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130302&page=08 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=2013030

http://hk.racing.nextmedia.com/fullresult.php?date=20130212&page=11
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130212&page=11 exist in cache
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=01 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=01
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=01 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=02
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=02 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=03
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=03 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=04
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=04 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130206&page=05
url = http://hk.racing.nextmedia.com/fullresult.php?date=2

http://hk.racing.nextmedia.com/fullresult.php?date=20130116&page=06
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130116&page=06 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130116&page=07
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130116&page=07 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130116&page=08
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130116&page=08 exist in cache
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130112&page=01 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130112&page=01
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130112&page=01 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130112&page=02
url = http://hk.racing.nextmedia.com/fullresult.php?date=20130112&page=02 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20130112&page=03
url = http://hk.racing.nextmedia.com/fullresult.php?date=2

...

http://hk.racing.nextmedia.com/fullresult.php?date=20111120&page=05
http://hk.racing.nextmedia.com/fullresult.php?date=20111120&page=06
http://hk.racing.nextmedia.com/fullresult.php?date=20111120&page=07
http://hk.racing.nextmedia.com/fullresult.php?date=20111120&page=08
http://hk.racing.nextmedia.com/fullresult.php?date=20111120&page=09
http://hk.racing.nextmedia.com/fullresult.php?date=20111120&page=10
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111116&page=01 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111116&page=01
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111116&page=01 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111116&page=02
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111116&page=02 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111116&page=03
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111116&page=03 exist in cache
http://hk.racing.nextmedia.com/f

http://hk.racing.nextmedia.com/fullresult.php?date=20111026&page=06
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111026&page=06 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111026&page=07
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111026&page=07 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111026&page=08
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111026&page=08 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111023&page=01
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111023&page=01 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111023&page=02
http://hk.racing.nextmedia.com/fullresult.php?date=20111023&page=03
http://hk.racing.nextmedia.com/fullresult.php?date=20111023&page=04
http://hk.racing.nextmedia.com/fullresult.php?date=20111023&page=05
http://hk.racing.nextmedia.com/fullresult.php?date=20111023&page=06
http://hk.racing.nextmedia.com/f

http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=04
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=04 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=05
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=05 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=06
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=06 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=07
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=07 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=08
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=08 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=09
url = http://hk.racing.nextmedia.com/fullresult.php?date=20111001&page=09 exist in cache
http://hk.racing.nextmedia.com/fullresult.php?date=2011100

http://hk.racing.nextmedia.com/fullresult.php?date=20110914&page=08
url = http://hk.racing.nextmedia.com/fullresult.php?date=20110914&page=08 exist in cache
exception = HTTP Error 503: Service Unavailable
exception = HTTP Error 503: Service Unavailable
exception = HTTP Error 503: Service Unavailable


Exception: Cannot open page http://hk.racing.nextmedia.com/fullresult.php?date=20110911&page=01

In [79]:
# hockey_web_columns = ['Place', 'HorseNo', 'HorseName', 'HorseCode', 'JockeyName', 'JockeyCode', 'TrainerName', 'TrianerCode',
#            'ActualWt', 'DeclarHorseWt', 'Draw', 'LBW', 'FinishTime', 'WinOdd', 'SecTime1', 'SecTime2', 'SecTime3',
#            'SecTime4', 'SecTime5', 'SecTime6']

In [80]:
# '名次	馬號	馬名	騎師	練馬師	實際負磅	排位體重	檔位	頭馬距離	沿途走位	完成時間	獨贏賠率'

In [81]:
# 'Plc.	Horse No.	Horse	Jockey	Trainer	Actual Wt.	Declar.Horse Wt.	Draw	LBW	Running Position	Finish Time	Win Odds'

In [82]:
# '1	7	SUPER FRESH(N194)	A Suborics	P F Yiu	123	1121	1	-	2	2	1	0.57.37	4.5'

In [83]:
# jackey_url = 'http://racing.hkjc.com/racing/Info/meeting/Results/english/Local/20130206/HV/5'

In [84]:
# 'http://bet.hkjc.com/racing/getXML.aspx?type=win&date=19-10-2016&venue=HV&raceno=1'